# LIME for NLP Interpretation

This notebook will use LIME, a ...., for interpreting natural language processing using spaCy.

https://medium.com/@lelwonger/using-lime-for-interpreting-nlp-c094703f98db

## Importing Modules

In [ ]:
pip install lime

In [3]:
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline

In [ ]:
forest = make_pipeline(tfidf, forest_model)
idx = 170
explainer = LimeTextExplainer(class_names=label.classes_)
exp_forest = explainer.explain_instance(data[idx], forest.predict_proba, labels = [0,1,2])
exp_forest.show_in_notebook(text = data[idx])

from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline

wrapped_model = KerasClassifier(build_fn = create_final_model, verbose = 1)

pipe = Pipeline([('tokenizer', custom_tokenizer()),
                 ('model', wrapped_model)])




Trying this with my data.

In [4]:
import pandas as pd

In [11]:
df = pd.read_csv('fake_real.csv')

In [12]:
df.head()

,title,text,subject,date,type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True


In [61]:
# create a smaller dataset
# Separate the DataFrame into two subsets based on categories
true_df = df[df['type'] == 'True']
fake_df = df[df['type'] == 'Fake']

# Randomly sample an equal number of rows from each subset
sampled_true = true_df.sample(n=20, replace=False, random_state=42)
sampled_fake = fake_df.sample(n=20, replace=False, random_state=42)

# Combine the sampled rows into a single DataFrame
sampled_df = pd.concat([sampled_true, sampled_fake])

# Reset the index of the sampled DataFrame
sampled_df.reset_index(drop=True, inplace=True)

In [ ]:
sampled_df.dropna()

In [66]:
sampled_df['text'] = pd.Series(df['text'], dtype="string")
sampled_df['type'] = pd.Series(df['type'], dtype="string")
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    40 non-null     object
 1   text     40 non-null     string
 2   subject  40 non-null     object
 3   date     40 non-null     object
 4   type     40 non-null     string
dtypes: object(3), string(2)
memory usage: 1.7+ KB


In [8]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

In [67]:
# Define a custom tokenizer using spaCy
def custom_tokenizer(text):
    return [token.lemma_ for token in nlp(text)]

# Load your DataFrame
# Assuming your DataFrame is named df
# Extract text data from the DataFrame
text_data = sampled_df['text'].tolist()

# Extract labels (if available)
# Assuming your labels are in a column named 'type'
labels = sampled_df['type'].tolist()

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)

# Create a random forest model
forest_model = RandomForestClassifier()

# Create a pipeline combining TF-IDF vectorization and random forest model
forest = make_pipeline(tfidf, forest_model)

In [68]:
# Train your random forest model on your text data and labels
# Assuming you have X_train and y_train
forest.fit(text_data, labels)

C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function custom_tokenizer at 0x0000023A836B32E0>)),
                ('randomforestclassifier', RandomForestClassifier())])

In [69]:
# Create LimeTextExplainer instance
explainer = LimeTextExplainer(class_names=forest.classes_)

In [22]:
print(text_data[1])

(Reuters) - In September 2014, Gilberto Velasquez, a 38-year-old house painter from El Salvador, received life-changing news: The U.S. government had decided to shelve its deportation action against him.  The move was part of a policy change initiated by then-President Barack Obama in 2011 to pull back from deporting immigrants who had formed deep ties in the United States and whom the government considered no threat to public safety. Instead, the administration would prioritize illegal immigrants who had committed serious crimes.  Last month, things changed again for the painter, who has lived in the United States illegally since 2005 and has a U.S.-born child. He received news that the government wanted to put his deportation case back on the court calendar, citing another shift in priorities, this time by President Donald Trump.  The Trump administration has moved to reopen the cases of hundreds of illegal immigrants who, like Velasquez, had been given a reprieve from deportation, a

In [70]:
# Explain instance using LimeTextExplainer
exp_forest = explainer.explain_instance(text_data[0], forest.predict_proba, labels=forest.classes_)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# Show explanation in notebook
exp_forest.show_in_notebook(text=text_data[idx])